In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import torch
from google.colab import files
import json

In [ ]:
# Upload file CSV
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df_input = pd.read_csv(filename)

Saving cosmetic_products_5000.csv to cosmetic_products_5000.csv


In [ ]:
# Load model dan tokenizer
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def batch_generate_ads(texts, max_new_tokens=50):
    prompts = [f"Write a concise and persuasive advertisement for the following product:\n\n{text}\n\nAd:" for text in texts]

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=30,
            top_p=0.90,
            temperature=0.7,
            num_return_sequences=1
        )

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
# Eksekusi per batch dan simpan hasil
batch_size = 32
results = []

for i in range(0, len(df_input), batch_size):
    batch_df = df_input.iloc[i:i+batch_size]
    descriptions = batch_df["ShortDescription"].tolist()
    ads = batch_generate_ads(descriptions)  # Pastikan fungsi ini sudah terdefinisi

    for idx, ad_text in enumerate(ads):
        results.append({
            "ProductName": batch_df.iloc[idx]["ProductName"],
            "description": descriptions[idx],
            "ad_text": ad_text
        })

    # Simpan hasil sementara setiap batch supaya aman kalau crash
    pd.DataFrame(results).to_csv("ads_generated_english_partial.csv", index=False)

# Simpan hasil final setelah semua batch selesai
df_output = pd.DataFrame(results)
df_output.to_csv("ads_generated_english.csv", index=False)

print(df_output.head())

                ProductName  \
0  Bright Exfoliating Scrub   
1              Velvet Toner   
2        Smooth Moisturizer   
3         Radiant Face Mask   
4         Bright Hair Serum   

                                         description  \
0  Reduces puffiness and dark circles with argan ...   
1  Provides rich color while nourishing with gree...   
2  Reduces puffiness and dark circles with essent...   
3  Repairs damaged hair and adds shine using shea...   
4  Delivers smooth coverage with a lightweight, n...   

                                             ad_text  
0        It's a great way to keep puffy circles out.  
1  The product is made with green tea extract and...  
2  Use this product as a cleanser to reduce puffi...  
3  The product is made of natural shea butter and...  
4  A product that delivers smooth coverage with a...  


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-id"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/796k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/291M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/291M [00:00<?, ?B/s]

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()
    # Bisa ganti ":" dengan koma atau hapus
    text = text.replace(":", ",")  # atau: text = text.replace(":", "")
    if not text.endswith("."):
        text += "."
    return text

In [ ]:
def translate_batch(texts):
    texts_filtered = [str(text).strip() if text == text else "" for text in texts]  # text == text untuk cek NaN
    texts_filtered = [text if text != "" else " " for text in texts_filtered]
    inputs = tokenizer(texts_filtered, return_tensors="pt", padding=True, truncation=True).to("cuda")
    translated = model.generate(**inputs, max_new_tokens=60)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
df = pd.read_csv("ads_generated_english.csv")  # File hasil generate iklan awal

In [ ]:
batch_size = 32

translations_ad = []
translations_desc = []

for i in range(0, len(df), batch_size):
    # Ambil dan bersihkan batch kolom iklan (ad_text)
    batch_ad_raw = df['ad_text'][i:i+batch_size]
    batch_ad = batch_ad_raw.fillna("").astype(str).apply(clean_text).tolist()

    # Ambil dan bersihkan batch kolom deskripsi (description)
    batch_desc_raw = df['description'][i:i+batch_size]
    batch_desc = batch_desc_raw.fillna("").astype(str).apply(clean_text).tolist()

    # Translate batch ad_text dengan penanganan error
    try:
        results_ad = translate_batch(batch_ad)
    except Exception as e:
        print(f"Error translating ad_text batch {i}: {e}")
        results_ad = [""] * len(batch_ad)
    translations_ad.extend(results_ad)

    # Translate batch description dengan penanganan error
    try:
        results_desc = translate_batch(batch_desc)
    except Exception as e:
        print(f"Error translating description batch {i}: {e}")
        results_desc = [""] * len(batch_desc)
    translations_desc.extend(results_desc)

# Simpan hasil ke kolom baru di dataframe
df['deskripsi_indonesia'] = translations_desc
df['iklan_indonesia'] = translations_ad

# Simpan ke file CSV
df.to_csv("ads_translated_indonesia.csv", index=False)

In [ ]:
print(df.head(5))

                ProductName  \
0  Bright Exfoliating Scrub   
1              Velvet Toner   
2        Smooth Moisturizer   
3         Radiant Face Mask   
4         Bright Hair Serum   

                                         description  \
0  Reduces puffiness and dark circles with argan ...   
1  Provides rich color while nourishing with gree...   
2  Reduces puffiness and dark circles with essent...   
3  Repairs damaged hair and adds shine using shea...   
4  Delivers smooth coverage with a lightweight, n...   

                                             ad_text  \
0        It's a great way to keep puffy circles out.   
1  The product is made with green tea extract and...   
2  Use this product as a cleanser to reduce puffi...   
3  The product is made of natural shea butter and...   
4  A product that delivers smooth coverage with a...   

                                 deskripsi_indonesia  \
0  Mengurangi puffiness dan lingkaran gelap denga...   
1  Menyediakan warna yang k

In [ ]:
df = df.drop(columns=['ad_text','description'])

In [ ]:
df.rename(columns={"ProductName": "nama_produk"}, inplace=True)

In [ ]:
df.to_csv("produk_kostemik_generate_ad.csv", index=False)

In [ ]:
df.head(5)

,nama_produk,deskripsi_indonesia,iklan_indonesia
0,Bright Exfoliating Scrub,Mengurangi puffiness dan lingkaran gelap denga...,Ini adalah cara yang bagus untuk menjaga lingk...
1,Velvet Toner,Menyediakan warna yang kaya sambil memelihara ...,Produk ini dibuat dengan ekstrak teh hijau dan...
2,Smooth Moisturizer,Mengurangi puffiness dan lingkaran gelap denga...,Gunakan produk ini sebagai pembersih untuk men...
3,Radiant Face Mask,Memperbaiki rambut yang rusak dan menambah ber...,Produknya terbuat dari mentega dan lilin lebah...
4,Bright Hair Serum,"Memberikan cakupan halus dengan ringan, selesa...",Sebuah produk yang memberikan cakupan halus de...


In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 4.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Misal df sudah ada
judul = "Generated Indonesian Advertisement Texts for Cosmetic Products"
kontak = "Kontak: wa.me/081273333600"

with pd.ExcelWriter("ads_terjemahan_dengan_judul.xlsx", engine="xlsxwriter") as writer:
    # Tulis df mulai baris 3 (index=2) supaya baris atas bisa dipakai judul
    df.to_excel(writer, sheet_name="Sheet1", startrow=2, index=False)

    workbook  = writer.book
    worksheet = writer.sheets["Sheet1"]

    # Tambah judul di baris 1 (index=0), kolom A (index=0)
    worksheet.write(0, 0, judul)

    # Tambah kontak di baris 1, kolom B (index=1)
    worksheet.write(0, 1, kontak)

In [ ]:
df_new = pd.read_excel("ads_terjemahan_dengan_judul.xlsx", sheet_name="Sheet1", header=2)
print(df_new.head())

                nama_produk  \
0  Bright Exfoliating Scrub   
1              Velvet Toner   
2        Smooth Moisturizer   
3         Radiant Face Mask   
4         Bright Hair Serum   

                                 deskripsi_indonesia  \
0  Mengurangi puffiness dan lingkaran gelap denga...   
1  Menyediakan warna yang kaya sambil memelihara ...   
2  Mengurangi puffiness dan lingkaran gelap denga...   
3  Memperbaiki rambut yang rusak dan menambah ber...   
4  Memberikan cakupan halus dengan ringan, selesa...   

                                     iklan_indonesia  
0  Ini adalah cara yang bagus untuk menjaga lingk...  
1  Produk ini dibuat dengan ekstrak teh hijau dan...  
2  Gunakan produk ini sebagai pembersih untuk men...  
3  Produknya terbuat dari mentega dan lilin lebah...  
4  Sebuah produk yang memberikan cakupan halus de...  
